# Exploring Data using SQL

This notebook loads the csv files into a SQLite database for easy querying using the Pandas and sqlite3 Python package.

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        f = os.path.join(dirname, filename)
        files.append(f)
        print(f)

# Any results you write to the current directory are saved as output.

/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv
/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv


## Converting all CSVs to a single DB

In [36]:
!pip install csv-to-sqlite

In [37]:
def makeExecutable(ls):
    executable = str(ls).replace(',', '').replace('[', '').replace(']','').replace(' ', ' -f ')
    return executable

files_str = makeExecutable(files)

!echo $files_str

/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv -f /kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv -f /kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv -f /kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv -f /kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv -f /kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv -f /kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv -f /kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv


In [38]:
!csv-to-sqlite -f $files_str -o 'coronavirus.db'

Error on table time_series_covid_19_confirmed_US: 
 table [time_series_covid_19_confirmed_US] already exists
Error on table COVID19_open_line_list: 
 duplicate column name: 
Error on table time_series_covid_19_recovered: 
 table [time_series_covid_19_recovered] already exists
Error on table time_series_covid_19_confirmed: 
 table [time_series_covid_19_confirmed] already exists
Error on table time_series_covid_19_deaths_US: 
 table [time_series_covid_19_deaths_US] already exists
Error on table COVID19_line_list_data: 
 duplicate column name: 
Error on table covid_19_data: 
 table [covid_19_data] already exists
Error on table time_series_covid_19_deaths: 
 table [time_series_covid_19_deaths] already exists
Written 0 rows into 8 tables in 0.013 seconds


### Opening the Database in Sqlite3

In [39]:
import sqlite3 
  
# connecting to the database 
def makeConnection(db_name):
    return sqlite3.connect(db_name)
    
# fetch all the tables from the db
def sql_fetch(con):

    cursorObj = connection.cursor()

    cursorObj.execute('SELECT name FROM sqlite_master WHERE TYPE="table"')

    print(cursorObj.fetchall())

print('Tables:')
connection = makeConnection("coronavirus.db")
sql_fetch(connection)

Tables:
[('covid_19_data',), ('time_series_covid_19_deaths_US',), ('time_series_covid_19_confirmed_US',), ('time_series_covid_19_confirmed',), ('time_series_covid_19_recovered',), ('time_series_covid_19_deaths',)]


### Check Database
Here we will use Pandas to run our SQL queries and display them in our notebook in a more interpreatable way than printing sqlite execution statements

In [40]:
pd.read_sql_query('SELECT * FROM time_series_covid_19_confirmed LIMIT 5', connection)

,Province_State,Country_Region,Lat,Long,1_22_20,1_23_20,1_24_20,1_25_20,1_26_20,1_27_20,...,3_22_20,3_23_20,3_24_20,3_25_20,3_26_20,3_27_20,3_28_20,3_29_20,3_30_20,3_31_20
0,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,40,40,74,84,94,110,110,120,170,174
1,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,89,104,123,146,174,186,197,212,223,243
2,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,201,230,264,302,367,409,454,511,584,716
3,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,113,133,164,188,224,267,308,334,370,376
4,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,3,3,3,4,4,5,7,7,7


## Reformatting the Columns
Unfortunately, sqlite3 cannot interpret column names with "**/**" characters. Here we use pandas to rename the columns to sql format. Fortunately this only needs to be done for 3 files:
* [/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv]()
* [/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv]()
* [/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv]()

Unfortunately, this has to be done outside of sqlite so we will have to recreate our database with the corrected csv files.

In [41]:
corupt_files = ["/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv",
                "/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv",
                "/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv"]

fixed_files = []

for file in corupt_files:
    df = pd.read_csv(file)
    col_map = {col:col.replace('/', '_') for col in df.columns}
    df = df.rename(columns=col_map)
    fixed = file[file.rindex('/')+1:]
    df.to_csv(fixed, index=False)
    fixed_files.append(fixed)

Verify the column names were changed correctly:

In [42]:
pd.read_csv(fixed).head()

,Province_State,Country_Region,Lat,Long,1_22_20,1_23_20,1_24_20,1_25_20,1_26_20,1_27_20,...,3_22_20,3_23_20,3_24_20,3_25_20,3_26_20,3_27_20,3_28_20,3_29_20,3_30_20,3_31_20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1,1,1,2,4,4,4,4,4,4
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2,4,5,5,6,8,10,10,11,15
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,17,17,19,21,25,26,29,31,35,44
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,1,1,1,1,3,3,3,6,8,12
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2,2


Consturct a new list with the repaired files

In [43]:
leftovers = list(set(files) - set(corupt_files))
new_files = leftovers + fixed_files
new_files_str = makeExecutable(new_files)
new_files_str

"'/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv' -f '/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv' -f '/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv' -f '/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths_US.csv' -f '/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed_US.csv' -f 'time_series_covid_19_confirmed.csv' -f 'time_series_covid_19_recovered.csv' -f 'time_series_covid_19_deaths.csv'"

Remove the original database and create a new one

In [44]:
! rm coronavirus.db
! csv-to-sqlite -f $new_files_str -o 'coronavirus.db'

Error on table COVID19_line_list_data: 
 duplicate column name: 
Error on table COVID19_open_line_list: 
 duplicate column name: 
Written 17931 rows into 8 tables in 0.326 seconds


Connect to the new database 

In [45]:
connection = makeConnection("coronavirus.db")

## Brief SQL Data Exploration

In [46]:
conf_df = pd.read_sql_query('SELECT * FROM time_series_covid_19_confirmed LIMIT 5', connection)
conf_df

,Province_State,Country_Region,Lat,Long,1_22_20,1_23_20,1_24_20,1_25_20,1_26_20,1_27_20,...,3_22_20,3_23_20,3_24_20,3_25_20,3_26_20,3_27_20,3_28_20,3_29_20,3_30_20,3_31_20
0,,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,40,40,74,84,94,110,110,120,170,174
1,,Albania,41.1533,20.1683,0,0,0,0,0,0,...,89,104,123,146,174,186,197,212,223,243
2,,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,201,230,264,302,367,409,454,511,584,716
3,,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,113,133,164,188,224,267,308,334,370,376
4,,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,3,3,3,4,4,5,7,7,7


### Today's Counts and Rankings
Determine the number of countries with cases on the most recent day. This requires us to get the last column in the dataframe which cannot be performed in SQL. Instead we will use python to obtain it and then pass the value to our SQL query.

#### Worldwide Death Toll

In [47]:
pd.read_sql_query('SELECT SUM("'+ df.columns[-1] +'") AS "Worldwide Death Toll"' +
                  ' FROM time_series_covid_19_deaths', 
                  connection)

,Worldwide Death Toll
0,42107


#### Top Ten Countries with Highest Death Tolls

In [48]:
pd.read_sql_query('SELECT Country_Region, "'+ df.columns[-1] +'" AS "Death Toll"' +
                  ' FROM time_series_covid_19_deaths' +
                  ' ORDER BY "' + df.columns[-1] + '" DESC' +
                  ' LIMIT 10', 
                  connection)

,Country_Region,Death Toll
0,Italy,12428
1,Spain,8464
2,US,3873
3,France,3523
4,China,3187
5,Iran,2898
6,United Kingdom,1789
7,Netherlands,1039
8,Germany,775
9,Belgium,705


#### Countries with Confirmed Cases

In [49]:
pd.read_sql_query('SELECT ROUND('+
                      '1.0 * SUM(' +
                          ' CASE WHEN "' + df.columns[-1] +'" > 0 THEN 1' +
                          ' ELSE 0 ' +
                          ' END) / COUNT("'+ df.columns[-1] +'"), 2)' +  
                          ' AS "Countries/Regions With Confirmed Cases (%)"' +
                  ' FROM time_series_covid_19_confirmed', 
                  connection)

,Countries/Regions With Confirmed Cases (%)
0,0.99


#### Countries with Most Cases and their Severity Score
Here we define the severity score as the death to case ratio, the more deaths per case, the more severe the virus's impact.

In [50]:
pd.read_sql_query(' WITH deaths AS (SELECT Country_Region, "'+ df.columns[-1] +'" AS "Deaths"' +
                  ' FROM time_series_covid_19_deaths),' +
                  ' cases AS (SELECT Country_Region, "'+ df.columns[-1] +'" AS "Cases"' +
                  ' FROM time_series_covid_19_confirmed cases)' +
                  ' SELECT cases.Country_Region, SUM(cases.Cases) AS "Cases", SUM(deaths.Deaths) AS "Deaths",' +
                  ' ROUND(1.0 * deaths.Deaths/cases.Cases, 2) AS "Severity Score"'
                  ' FROM cases'
                  ' JOIN deaths' +
                  ' ON cases.Country_Region = deaths.Country_Region' +
                  ' GROUP BY cases.Country_Region'
                  ' ORDER BY cases.Cases DESC' + 
                  ' LIMIT 10',
                  connection)

,Country_Region,Cases,Deaths,Severity Score
0,US,188172,3873,0.02
1,Italy,105792,12428,0.12
2,Spain,95923,8464,0.09
3,Germany,71808,775,0.01
4,Iran,44605,2898,0.06
5,Switzerland,16605,433,0.03
6,Turkey,13531,214,0.02
7,Belgium,12775,705,0.06
8,Austria,10180,128,0.01
9,"Korea, South",9786,162,0.02


#### Countries with Least Cases

In [51]:
pd.read_sql_query('SELECT Country_Region, SUM("'+ df.columns[-1] +'") AS "Cases"'
                  ' FROM time_series_covid_19_confirmed' +
                  ' GROUP BY Country_Region' +
                  ' ORDER BY "'+ df.columns[-1] +'"' +
                  ' LIMIT 10',
                  connection)

,Country_Region,Cases
0,Papua New Guinea,1
1,Saint Vincent and the Grenadines,1
2,Sierra Leone,1
3,Timor-Leste,1
4,Burundi,2
5,MS Zaandam,2
6,Belize,3
7,Central African Republic,3
8,Liberia,3
9,Bhutan,4


#### Average Number of Cases per Country

In [62]:
pd.read_sql_query('WITH total_cases AS ('
                  ' SELECT SUM("'+ df.columns[-1] +'") AS "Cases"' +
                  ' FROM time_series_covid_19_confirmed' +
                  ' GROUP BY Country_Region)' +
                  ' SELECT ROUND(AVG(Cases)) AS "Average Cases per Country"' +
                  ' FROM total_cases',
                  connection)

,Average Cases per Country
0,4763.816667
